In [ ]:
pip install pandas
pip install plotly

In [1]:
import ccxt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime as dt
import plotly.graph_objects as go
from datetime import timedelta  
import plotly.io as pio
import requests
from itertools import combinations
import time
import ast
# from ipyparallel import Client


In [2]:
binance = ccxt.binance()
okx = ccxt.okex()
bitstamp = ccxt.bitstamp()
bitfinex = ccxt.bitfinex()
kraken = ccxt.kraken()


In [3]:

class Orderbook:
    def __init__(self, symbol, exchange):
        self.symbol = symbol
        self.exchange = exchange

    @staticmethod
    def unix_milliseconds_to_datetime(unix_milliseconds):
        unix_seconds = unix_milliseconds / 1000.0
        return dt.utcfromtimestamp(unix_seconds)
        
    def get_ob(self, duration, interval):
        symbols, bids, asks, datetimes, timestamps = [], [], [], [], []

        num_iterations = int(round(duration/interval))

        for i in range(num_iterations):
            try:
                ob = self.exchange.fetch_order_book(self.symbol, limit=1)
            except:
                time.sleep(interval*60)
                ob = self.exchange.fetch_order_book(self.symbol, limit=1)

            if ob['timestamp'] is None:
                timestamps.append(None)
            else:
                timestamps.append(self.unix_milliseconds_to_datetime(ob['timestamp']))

            bids.append(ob['bids'][0] if ob['bids'] else None)
            asks.append(ob['asks'][0] if ob['asks'] else None)
            symbols.append(ob['symbol'])
            datetimes.append(dt.now())
            print(i)
            time.sleep(interval*60)

        dic = {'Symbol': symbols, 'Bid': bids, 'Ask': asks, 'Datetime': datetimes, 'Timestamp': timestamps}   
        df = pd.DataFrame(dic)
        return df


In [ ]:
btc_bin = Orderbook('BTC/USDT',binance)
# df = btc_bin.get_ob(60,1)

In [ ]:
bin_ob = pd.read_csv('BIN_OB.csv')
bin_ob['Exchange'] = 'Binance'
okx_ob = pd.read_csv('OKX_OB.csv')
okx_ob['Exchange'] = 'OKX'
bts_ob = pd.read_csv('BST_OB.csv')
bts_ob['Exchange'] = 'Bitstamp'
bfx_ob = pd.read_csv('BFX_OB.csv')
bfx_ob['Exchange'] = 'Bitfinex'
kra_ob = pd.read_csv('KRA_OB.csv')
kra_ob['Exchange'] = 'Kraken'



In [ ]:

OBs = [bin_ob, okx_ob, bts_ob, bfx_ob, kra_ob]

for i in OBs:
    i['Datetime'] = pd.to_datetime(i['Datetime'])
    i['Time'] = i['Datetime'].apply(lambda x: x.time())
    i['Time'] = i['Time'].apply(lambda x: x.replace(second=0, microsecond=0))
    i.drop(['Datetime','Timestamp'], axis=1, inplace=True)  # Remove the original 'Datetime' column
    # i.rename(columns={'Time': 'Datetime'}, inplace=True)  # Rename the 'Time' column to 'Datetime'
    i = i[['Symbol', 'Bid', 'Ask', 'Time']]  # Select specific columns


# master_ob = pd.concat(OBs,ignore_index=True)
# master_ob = master_ob.set_index(['Time', 'Exchange'], inplace=True)
# master_ob = master_ob.sort_index()
# master_ob.to_csv('Master_ob.csv')


In [276]:
master_ob = pd.read_csv('Master_ob.csv')
master_ob['Bid'] = master_ob['Bid'].apply(ast.literal_eval) 
master_ob['Ask'] = master_ob['Ask'].apply(ast.literal_eval)

In [277]:
master_ob

,Time,Exchange,Symbol,Bid,Ask
0,16:37:00,Binance,BTC/USDT,"[25907.73, 16.84722]","[25907.74, 8.16039]"
1,16:37:00,Bitfinex,BTC/USDT,"[25899.0, 0.13736287]","[25900.0, 0.1192]"
2,16:37:00,Bitstamp,BTC/USDT,"[25901.0, 0.0144]","[25911.0, 0.23366829]"
3,16:37:00,OKX,BTC/USDT,"[25901.4, 0.56009766]","[25901.5, 0.30131139]"
4,16:38:00,Binance,BTC/USDT,"[25891.92, 2.65783]","[25891.93, 17.09659]"
...,...,...,...,...,...
295,17:37:00,Kraken,BTC/USDT,"[25878.7, 0.001, 1693586225]","[25880.9, 0.725, 1693586223]"
296,17:37:00,OKX,BTC/USDT,"[25877.0, 0.24792826]","[25877.1, 1.3057301]"
297,17:38:00,Binance,BTC/USDT,"[25877.25, 3.0572]","[25877.26, 8.72116]"
298,17:38:00,Kraken,BTC/USDT,"[25875.9, 0.187, 1693586278]","[25876.7, 0.023, 1693586284]"


In [278]:
bid, bid_size, ask, ask_size = [], [], [], []

for i in master_ob['Bid']:
    bid.append(i[0])
    bid_size.append(i[1]*i[0])

for j in master_ob['Ask']:
    ask.append(j[0])
    ask_size.append(j[1]*j[0])

master_ob['Bid'] = bid
master_ob['Bid Size($)'] = bid_size
master_ob['Ask'] = ask
master_ob['Ask Size($)'] = ask_size

In [279]:
# df = master_ob[master_ob['Time'] == '16:43:00']

In [280]:
master_ob

,Time,Exchange,Symbol,Bid,Ask,Bid Size($),Ask Size($)
0,16:37:00,Binance,BTC/USDT,25907.73,25907.74,436473.227011,211417.262419
1,16:37:00,Bitfinex,BTC/USDT,25899.00,25900.00,3557.560970,3087.280000
2,16:37:00,Bitstamp,BTC/USDT,25901.00,25911.00,372.974400,6054.579062
3,16:37:00,OKX,BTC/USDT,25901.40,25901.50,14507.313531,7804.416968
4,16:38:00,Binance,BTC/USDT,25891.92,25891.93,68816.321734,442663.711519
...,...,...,...,...,...,...,...
295,17:37:00,Kraken,BTC/USDT,25878.70,25880.90,25.878700,18763.652500
296,17:37:00,OKX,BTC/USDT,25877.00,25877.10,6415.639584,33788.508371
297,17:38:00,Binance,BTC/USDT,25877.25,25877.26,79111.928700,225679.724822
298,17:38:00,Kraken,BTC/USDT,25875.90,25876.70,4838.793300,595.164100


In [281]:
def best_arb(df):
    # Find the index of the lowest Ask for each time group
    lowest_ask_index = df.groupby('Time')['Ask'].idxmin()
    highest_bid_index = df.groupby('Time')['Bid'].idxmax()

    # Use the index to get the corresponding rows
    lowest_ask_rows = df.loc[lowest_ask_index]
    highest_bid_rows = df.loc[highest_bid_index]

    # Get the best pair of exchanges
    ex_pair = [lowest_ask_rows['Exchange'].iloc[0],highest_bid_rows['Exchange'].iloc[0]]

    # Get the minimum 'Ask Size($)' and maximum 'Bid Size($)'
    min_ask_size = lowest_ask_rows['Ask Size($)'].iloc[0]
    max_bid_size = highest_bid_rows['Bid Size($)'].iloc[0]

    if min_ask_size <= max_bid_size:
        buy_pos = min_ask_size
        no_tokens = (min_ask_size / lowest_ask_rows['Ask'].iloc[0])
    else:
        buy_pos = max_bid_size
        no_tokens = (max_bid_size / highest_bid_rows['Bid'].iloc[0])

    sell_pos = no_tokens*highest_bid_rows['Bid'].iloc[0]

    lowest_ask = lowest_ask_rows['Ask'].iloc[0]
    highest_bid = highest_bid_rows['Bid'].iloc[0]
    

    # return lowest_ask_rows,highest_bid_rows
    return no_tokens,ex_pair,buy_pos,sell_pos,lowest_ask,highest_bid

# Call the function with your DataFrame
# best_arb(df)



In [291]:
arb = master_ob.groupby('Time').apply(best_arb)


In [292]:
list_of_dicts = [{'Time': index, 'No. of Tokens': val[0], 'Best Buy/Sell': val[1], 'Buy Position($)': val[2],'Sell Position($)':val[3],'Lowest Ask':val[4],'Highest Bid':val[5]} for index, val in arb.items()]

In [293]:
df = pd.DataFrame(list_of_dicts)
df

,Time,No. of Tokens,Best Buy/Sell,Buy Position($),Sell Position($),Lowest Ask,Highest Bid
0,16:37:00,0.119200,"[Bitfinex, Binance]",3087.280000,3088.201416,25900.00,25907.73
1,16:38:00,0.012982,"[Binance, Bitfinex]",336.258437,336.258437,25891.93,25901.00
2,16:39:00,0.000579,"[Bitfinex, Binance]",15.001245,15.004994,25891.00,25897.47
3,16:40:00,0.001929,"[Bitstamp, OKX]",49.954420,49.954420,25889.00,25896.00
4,16:41:00,0.001000,"[Bitfinex, Kraken]",25.878200,25.878200,25874.00,25878.20
...,...,...,...,...,...,...,...
58,17:35:00,0.159292,"[Bitstamp, Binance]",4127.403093,4129.292291,25911.00,25922.86
59,17:36:00,0.001678,"[Bitstamp, Binance]",43.414994,43.462586,25880.00,25908.37
60,17:37:00,0.244530,"[OKX, Binance]",6328.812976,6328.812976,25877.10,25881.54
61,17:38:00,0.023000,"[Kraken, Binance]",595.164100,595.176750,25876.70,25877.25


In [295]:
df['Percentage Gain'] = (df['Sell Position($)'] - df['Buy Position($)'])/df['Buy Position($)']
df['Dollar Gain']=(df['Sell Position($)'] - df['Buy Position($)'])
# df.to_csv('BTC_ARB.csv')

In [288]:
df

,Time,No. of Tokens,Best Buy/Sell,Buy Position,Sell Position,Lowest Ask,Highest Bid,Percentage Gain,Dollar Gain
0,16:37:00,0.119200,"[Bitfinex, Binance]",3087.280000,3088.201416,25900.00,25907.73,0.000298,0.921416
1,16:38:00,0.012982,"[Binance, Bitfinex]",336.258437,336.258437,25891.93,25901.00,0.000000,0.000000
2,16:39:00,0.000579,"[Bitfinex, Binance]",15.001245,15.004994,25891.00,25897.47,0.000250,0.003749
3,16:40:00,0.001929,"[Bitstamp, OKX]",49.954420,49.954420,25889.00,25896.00,0.000000,0.000000
4,16:41:00,0.001000,"[Bitfinex, Kraken]",25.878200,25.878200,25874.00,25878.20,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...
58,17:35:00,0.159292,"[Bitstamp, Binance]",4127.403093,4129.292291,25911.00,25922.86,0.000458,1.889198
59,17:36:00,0.001678,"[Bitstamp, Binance]",43.414994,43.462586,25880.00,25908.37,0.001096,0.047592
60,17:37:00,0.244530,"[OKX, Binance]",6328.812976,6328.812976,25877.10,25881.54,0.000000,0.000000
61,17:38:00,0.023000,"[Kraken, Binance]",595.164100,595.176750,25876.70,25877.25,0.000021,0.012650


In [299]:
df = df[['Time','Best Buy/Sell','Lowest Ask','Highest Bid','No. of Tokens','Buy Position($)','Sell Position($)','Percentage Gain','Dollar Gain']]
df.to_csv('BTC_ARB2.csv')

In [300]:
df

,Time,Best Buy/Sell,Lowest Ask,Highest Bid,No. of Tokens,Buy Position($),Sell Position($),Percentage Gain,Dollar Gain
0,16:37:00,"[Bitfinex, Binance]",25900.00,25907.73,0.119200,3087.280000,3088.201416,0.000298,0.921416
1,16:38:00,"[Binance, Bitfinex]",25891.93,25901.00,0.012982,336.258437,336.258437,0.000000,0.000000
2,16:39:00,"[Bitfinex, Binance]",25891.00,25897.47,0.000579,15.001245,15.004994,0.000250,0.003749
3,16:40:00,"[Bitstamp, OKX]",25889.00,25896.00,0.001929,49.954420,49.954420,0.000000,0.000000
4,16:41:00,"[Bitfinex, Kraken]",25874.00,25878.20,0.001000,25.878200,25.878200,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...
58,17:35:00,"[Bitstamp, Binance]",25911.00,25922.86,0.159292,4127.403093,4129.292291,0.000458,1.889198
59,17:36:00,"[Bitstamp, Binance]",25880.00,25908.37,0.001678,43.414994,43.462586,0.001096,0.047592
60,17:37:00,"[OKX, Binance]",25877.10,25881.54,0.244530,6328.812976,6328.812976,0.000000,0.000000
61,17:38:00,"[Kraken, Binance]",25876.70,25877.25,0.023000,595.164100,595.176750,0.000021,0.012650
